Para empezar a seleccionar los datos primero tenemos que leerlos. Para esto se usa la libreria de `pandas` para leer la base de datos. Luego imprimimos algunos datos para revisar si podría existir una relacion.

In [10]:
import pandas as pd
data = pd.read_csv("Vino_Tinto.csv")
print("Dimensiones:", data.shape)
print("Primeras 5 filas de datos:")
print(data.head())

Dimensiones: (1599, 12)
Primeras 5 filas de datos:
   acidezFija  acidezVolatil  acidoCitrico  azucarResidual  cloruros  \
0         7.4           0.70          0.00             1.9     0.076   
1         7.8           0.88          0.00             2.6     0.098   
2         7.8           0.76          0.04             2.3     0.092   
3        11.2           0.28          0.56             1.9     0.075   
4         7.4           0.70          0.00             1.9     0.076   

   dioxidoAzufreLibre  dioxidoAzufreTotal  densidad    pH  sulfatos  alcohol  \
0                11.0                34.0    0.9978  3.51      0.56      9.4   
1                25.0                67.0    0.9968  3.20      0.68      9.8   
2                15.0                54.0    0.9970  3.26      0.65      9.8   
3                17.0                60.0    0.9980  3.16      0.58      9.8   
4                11.0                34.0    0.9978  3.51      0.56      9.4   

   calidad  
0        5  
1        

Ahora tenemos que separar los datos de entrenamiento y de prueba de la base de datos. Para este problema usaremos una separación 80/20, donde el 80% son los datos de entrenamiento y el 20% son los datos de prueba. Estos datos se deben elegir aleatoriamente, para esto se usa la funcion `train_test_split`. Recordemos que queremos saber que causa el cambio en calidad, por esto nuestra variable Y será la calidad.

In [11]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, train_size = 0.8)
X_train = train.drop('calidad', axis = 1)
X_test = test.drop('calidad', axis = 1)
Y_train = train.calidad
Y_test = test.calidad

Ya con los datos separados podemos proceder a la selección de variables. En el siguiente código realizaremos una selección hacia adelante. Para esto utilizaremos dos funciones: `LinearRegression` y `SequentialFeatureSelector`. La primera nos permite crear un estimador de regresión lineal, mientras que la segunda se encarga de ejecutar el proceso de selección de características.

En la función `SequentialFeatureSelector` definimos varios parámetros: el estimador (nuestro modelo de regresión lineal), el rango de características a elegir (`k_features`), la opción de selección hacia adelante (`forward=True`), la métrica de evaluación que en este caso será $R^2$ (`scoring="r2"`) y una validación cruzada de 5 folds (`cv=5`).

Finalmente, ajustamos este selector con nuestros datos de entrenamiento y mostramos los nombres de las mejores características seleccionadas.

In [12]:
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
estimator = LinearRegression()
sfs = SequentialFeatureSelector(
    estimator=estimator,
    k_features=(2, 8),
    forward=True,
    scoring="r2",
    cv=5,
)
sfs = sfs.fit(X_train, Y_train)
print("Mejores características hacia delante (nombres):", sfs.k_feature_names_)

Mejores características hacia delante (nombres): ('acidezVolatil', 'azucarResidual', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol')


Ahora evaluaremos la calidad del mejor modelo obtenido mediante la selección hacia adelante. Para esto, utilizaremos la combinación de variables elegidas tanto en los datos de entrenamiento como en los de prueba. Posteriormente, entrenamos nuestro modelo con la variable de salida en el conjunto de entrenamiento y realizamos las predicciones sobre el conjunto de prueba.

Finalmente, importamos la función `r2_score` para calcular el valor de $R^2$, que nos permitirá medir qué tan bien se ajusta el modelo a los datos. Por último, imprimimos este resultado.

In [13]:
X_train_selected = X_train[list(sfs.k_feature_names_)]
X_test_selected = X_test[list(sfs.k_feature_names_)]

estimator.fit(X_train_selected, Y_train)
Y_pred = estimator.predict(X_test_selected)

from sklearn.metrics import r2_score
r2_1 = r2_score(Y_test, Y_pred)
print("R^2 del modelo con las variables seleccionadas:", r2)

R^2 del modelo con las variables seleccionadas: 0.3229070119862125


Ahora repetiremos el mismo procedimiento que con la selección hacia adelante, pero esta vez modificaremos un parámetro: forward=False. Con este cambio realizaremos la selección hacia atrás, lo que nos permitirá eliminar variables poco relevantes y comparar posteriormente cuál método ofrece un mejor modelo.

En este caso, también definimos el rango de características a elegir, el estimador de regresión lineal, la métrica de evaluación ($R^2$) y una validación cruzada de 5 folds. Finalmente, ajustamos el selector con los datos de entrenamiento e imprimimos los nombres de las características seleccionadas.

In [14]:
sfs = SequentialFeatureSelector(
    estimator=estimator,
    k_features=(2, 5),
    forward=False,
    scoring="r2",
    cv=5,
)

sfs = sfs.fit(X_train, Y_train)
print("Mejores características hacia atras (nombres):", sfs.k_feature_names_)

Mejores características hacia atras (nombres): ('acidezVolatil', 'cloruros', 'dioxidoAzufreTotal', 'sulfatos', 'alcohol')


Igual que con la seleccion hacia delante obtenemos la R^2 con el mismo procedimiento. Y calcularemos la diferencia entre los modelos para saber cual fue mejor.

In [15]:
X_train_selected = X_train[list(sfs.k_feature_names_)]
X_test_selected = X_test[list(sfs.k_feature_names_)]

estimator.fit(X_train_selected, Y_train)
Y_pred = estimator.predict(X_test_selected)

from sklearn.metrics import r2_score
r2_2 = r2_score(Y_test, Y_pred)
print("R^2 del modelo con las variables seleccionadas:", r2)
print()

if r2_1 > r2_2:
    print("La seleccion hacia adelante tiene mejor calidad")
    print("Diferencia entre los dos modelos:", r2_1 - r2_2)
else:
    print("La seleccion hacia atras tiene mejor calidad")
    print("Diferencia entre los dos modelos:", r2_2 - r2_1)

R^2 del modelo con las variables seleccionadas: 0.3229070119862125

La seleccion hacia atras tiene mejor calidad
Diferencia entre los dos modelos: 0.007373855793524453
